In [1]:
import numpy as np
import itertools
import pickle

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch

EPOCH = 200
KERNEL_SIZE = 3
POOLING_SIZE = 2
BATCH_SIZE = 10

DATA_PATH = "./mit_data/"

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')


def list_to_list(input_list):
    input_list_to_list = list(itertools.chain(*input_list))
    return input_list_to_list

In [2]:
# Dataload part
le = preprocessing.LabelEncoder()

record_list = []
pickle_input = dict()
X, y = [], []

print("[INFO] Read records file from ", DATA_PATH)
with open(DATA_PATH + 'RECORDS') as f:
    record_lines = f.readlines()

for i in range(len(record_lines)):
    record_list.append(str(record_lines[i].strip()))

# for i in range(len(record_list)):
#     temp_path = DATA_PATH + "mit" + record_list[i] + ".pkl"
#     with open(temp_path, 'rb') as f:
#         pickle_input = pickle.load(f)
#         X.append(pickle_input[0][i])
#         print(pickle_input[0])
    
#     for j in range(len(pickle_input[1][i])):
#         check_ann = pickle_input[1][i]
#         temp_ann_list = list()
#         if check_ann == "N":            # Normal
#             temp_ann_list.append(0)

#         elif check_ann == "S":          # Supra-ventricular
#             temp_ann_list.append(1)

#         elif check_ann == "V":          # Ventricular
#             temp_ann_list.append(2)

#         elif check_ann == "F":          # False alarm
#             temp_ann_list.append(3)

#         else:                           # Unclassed 
#             temp_ann_list.append(4)

#         y.append(temp_ann_list)

for i in range(len(record_list)):
    temp_path = DATA_PATH + "mit" + record_list[i] + ".pkl"
    with open(temp_path, 'rb') as f:
        pickle_input = pickle.load(f)
        for i in range(len(pickle_input[0])):
            X.append(pickle_input[0][i])

        for i in range(len(pickle_input[1])):
            check_ann = pickle_input[1][i]
            temp_ann_list = list()
            if check_ann == "N":            # Normal
                temp_ann_list.append(0)

            elif check_ann == "S":          # Supra-ventricular
                temp_ann_list.append(1)

            elif check_ann == "V":          # Ventricular
                temp_ann_list.append(2)

            elif check_ann == "F":          # False alarm
                temp_ann_list.append(3)

            else:                           # Unclassed 
                temp_ann_list.append(4)
            y.append(temp_ann_list)

print("[SIZE]\t\tX length : {}\n\t\ty length : {}".format(len(X), len(y)))

npx = np.array(X)
npy = np.array(y)
print(npx.shape, npy.shape)

[INFO] Read records file from  ./mit_data/
[SIZE]		X length : 112599
		y length : 112599
(112599, 428) (112599, 1)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True
)

print("[SIZE]\t\tTrain X size : {}, Train y size : {}\n\t\tTest X size : {}, Test y size : {}"\
        .format(len(X_train), len(y_train), len(X_test), len(y_test)))

train_dataloadloader = DataLoader(X_train)
test_dataloader = DataLoader(X_test)

[SIZE]		Train X size : 78819, Train y size : 78819
		Test X size : 33780, Test y size : 33780


In [14]:
# Init
loss_fun = nn.MSELoss().to(device=device)
criterion = CrossEntropyLoss().to(device=device)

# weight of size [32, 428, 9], expected input[1, 78819, 428]


# First Part
conv1 = nn.Conv1d(in_channels=428, out_channels=78819, kernel_size=3)
bn1 = nn.BatchNorm1d(32)
relu1 = nn.ReLU()

# Second Part
conv2a = nn.Conv1d(in_channels=428, out_channels=64, kernel_size=KERNEL_SIZE)
conv2b = nn.Conv1d(in_channels=93, out_channels=64, kernel_size=KERNEL_SIZE)
bn2 = nn.BatchNorm1d(64)
relu2 = nn.ReLU()
maxpool2 = nn.MaxPool1d(64, stride=POOLING_SIZE)

# Third part
conv3a = nn.Conv1d(in_channels=47, out_channels=64, kernel_size=KERNEL_SIZE)
conv3b = nn.Conv1d(in_channels=47, out_channels=64, kernel_size=KERNEL_SIZE)
bn3 = nn.BatchNorm1d(64)
relu3 = nn.ReLU()
maxpool3 = nn.MaxPool1d(64, stride=POOLING_SIZE)

# Fourth part
conv4a = nn.Conv1d(in_channels=24, out_channels=128, kernel_size=KERNEL_SIZE)
conv4b = nn.Conv1d(in_channels=24, out_channels=128, kernel_size=KERNEL_SIZE)
bn4 = nn.BatchNorm1d(128)
relu4 = nn.ReLU()
maxpool4 = nn.MaxPool1d(128, stride=POOLING_SIZE)

# Fiveth part
conv5a = nn.Conv1d(in_channels=12, out_channels=128, kernel_size=KERNEL_SIZE)
conv5b = nn.Conv1d(in_channels=12, out_channels=128, kernel_size=KERNEL_SIZE)
bn5 = nn.BatchNorm1d(128)
relu5 = nn.ReLU()
maxpool5 = nn.MaxPool1d(128, stride=POOLING_SIZE)

# Sixth part
conv6a = nn.Conv1d(in_channels=6, out_channels=256, kernel_size=KERNEL_SIZE)
conv6b = nn.Conv1d(in_channels=6, out_channels=256, kernel_size=KERNEL_SIZE)
bn6 = nn.BatchNorm1d(256)
relu6 = nn.ReLU()
maxpool6 = nn.MaxPool1d(256, stride=POOLING_SIZE)

# Seventh part
conv7a = nn.Conv1d(in_channels=3, out_channels=256, kernel_size=KERNEL_SIZE)
conv7b = nn.Conv1d(in_channels=3, out_channels=256, kernel_size=KERNEL_SIZE)
bn7 = nn.BatchNorm1d(256)
relu7 = nn.ReLU()
maxpool7 = nn.MaxPool1d(256, stride=POOLING_SIZE)

first_input_layer = nn.Sequential(
    conv1, bn1, relu1
)

LFEM1 = nn.Sequential(
    conv2a, conv2b, bn2, relu2, maxpool2
)

LFEM2 = nn.Sequential(
    conv3a, conv3b, bn3, relu3, maxpool3
)

LFEM3 = nn.Sequential(
    conv4a, conv4b, bn4, relu4, maxpool4
)

LFEM4 = nn.Sequential(
    conv5a, conv5b, bn5, relu5, maxpool5
)

LFEM5 = nn.Sequential(
    conv6a, conv6b, bn6, relu6, maxpool6
)

LFEM6 = nn.Sequential(
    conv7a, conv7b, bn7, relu7, maxpool7
)

# Optim
first_optimizer = optim.Adam(first_input_layer.parameters())
lfem1_optimizer = optim.Adam(LFEM1.parameters())
lfem2_optimizer = optim.Adam(LFEM2.parameters())
lfem3_optimizer = optim.Adam(LFEM3.parameters())
lfem4_optimizer = optim.Adam(LFEM4.parameters())
lfem5_optimizer = optim.Adam(LFEM5.parameters())
lfem6_optimizer = optim.Adam(LFEM6.parameters())

for epcoh in range(EPOCH):
    first_input_layer.train()
    LFEM1.train()
    LFEM2.train()
    LFEM3.train()
    LFEM4.train()
    LFEM5.train()
    LFEM6.train()

In [26]:
X_train_ten, y_train_ten = Variable(torch.tensor(X_train, dtype=torch.float32).view(32, 428, -1)), Variable(torch.tensor(y_train, dtype=torch.float32, requires_grad=True))
print(X_train_ten[0].size())

conv1 = nn.Conv1d(in_channels=428, out_channels=32, kernel_size=3)
bn1 = nn.BatchNorm1d(32)
relu1 = nn.ReLU()

output = conv1(X_train_ten)
print(output.size())
print(output)

output = bn1(output)
print(output.size())
print(output)

output = relu1(output)
print(output.size())
print(output)

RuntimeError: shape '[32, 428, -1]' is invalid for input of size 33734532

In [18]:
for epoch in range(EPOCH):
        
        #  weight of size [32, 428, 9], expected input[1, 78819, 428]
        
        # Load Dataset
        X_train_ten, y_train_ten = Variable(torch.tensor(X_train, dtype=torch.float32).view(-1, 428, 3)), Variable(torch.tensor(y_train, dtype=torch.float32, requires_grad=True))
        print(X_train_ten.size())
        
        X_train_ten.to(device=device)
        y_train_ten.to(device=device)
        first_input_layer.eval()

        # Main Model Code
        # First Layer
        first_optimizer.zero_grad()
        first_output = first_input_layer(X_train_ten)
        first_output = first_output

        ''' ISSUE PART '''
        # Second Layer
        LFEM1.zero_grad()
        lfem1_output = LFEM1(first_output)
        lfem1_loss = criterion(lfem1_output, first_output)
        lfem1_loss.backward()
        lfem1_optimizer.step()

        # Third Layer
        LFEM2.zero_grad()
        lfem2_output = LFEM2(lfem2_output)
        lfem2_loss = criterion(lfem2_output, lfem1_output)
        lfem2_loss.backward()
        lfem2_optimizer.step()

        # Fourth Layer
        LFEM3.zero_grad()
        lfem3_output = LFEM3(lfem3_output)
        lfem3_loss = criterion(lfem3_output, lfem2_output)
        lfem3_loss.backward()
        lfem3_optimizer.step()

        # Fiveth Layer
        LFEM4.zero_grad()
        lfem4_output = LFEM4(lfem4_output)
        lfem4_loss = criterion(lfem4_output, lfem3_output)
        lfem4_loss.backward()
        lfem4_optimizer.step()

        # Sixten Layer
        LFEM5.zero_grad()
        lfem5_output = LFEM5(lfem4_output)
        lfem5_loss = criterion(lfem5_output, lfem4_output)
        lfem5_loss.backward()
        lfem5_optimizer.step()

        # Seventh Layer
        LFEM6.zero_grad()
        lfem6_output = LFEM6(lfem5_output)
        lfem6_loss = criterion(lfem6_output, lfem5_output)
        lfem6_loss.backward()
        lfem6_optimizer.step()

        print("Epoch : {}".format(epoch))

torch.Size([26273, 428, 3])
torch.Size([26273, 428, 3])


RuntimeError: running_mean should contain 78819 elements not 32